In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase
import time
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS

import matplotlib
import matplotlib as mpl

import matplotlib.pyplot as plt

import seaborn as sns

van Der Waals correction will be unavailable. Please install ased3


In [2]:
dir = '/scratch/Research/gutzwiller_research/GutzTestFiles/'

# Molecule file
molfile = dir + 'ts-0000.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol.xyz'

T = 300.0 # Temperature
dt = 10.0
C = 1.5 # Optimization convergence
steps = 40000

ntdir = '/nh/nest/u/jsmith/Research/gutzwiller_research/DarwinSets/GutzTrain/train_test_U2/U2_co3/'
cns = ntdir + 'rX-6.0A_32-4.0A_a8-8.params'
sae = ntdir + 'sae.dat'
nnf = ntdir + 'train'
Nn = 5

In [3]:
# Load molecule
mol = read(molfile)
L = 14.140479202201726
mol.set_cell(([[L, 0, 0],
               [0, L, 0],
               [0, 0, L]]))

mol.set_pbc((True, True, True))

In [4]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

In [5]:
def erg_rsix (Rmag):
    return np.power(Rmag,-6)

def frc_rsix (Rmag,R):
    F = 6.0*np.power(Rmag,-8)*(R)
    return -F

mol.calc.add_pairwise(erg_rsix, frc_rsix)

AttributeError: 'ANIENS' object has no attribute 'add_pairwise'

In [ ]:
# Optimize molecule
start_time = time.time()
dyn = QuasiNewton(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(mol, 0.01 * units.fs)  # 5 fs time step.

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%3.0fK)  '
          'Etot = %.3f' ' StdDev = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
dyn.attach(storeenergy, interval=1)

In [ ]:
start_time = time.time()
dyn.run(1000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')